In [7]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import alpaca_trade_api as alpaca
import random

In [13]:
from dotenv import load_dotenv
import os

# Load variables from .env into the environment
load_dotenv()

True

In [15]:
# CONNECT TO ALPACA AND RETRIEVE DATA
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
BASE_URL = 'https://paper-api.alpaca.markets/'

In [17]:
api = alpaca.REST(API_KEY, SECRET_KEY, BASE_URL)
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA31KUOTG0SB',
    'accrued_fees': '0',
    'admin_configurations': {},
    'balance_asof': '2025-03-28',
    'bod_dtbp': '332807.56',
    'buying_power': '332807.56',
    'cash': '179101.5',
    'created_at': '2025-03-20T22:56:44.15277Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 1,
    'currency': 'USD',
    'daytrade_count': 8,
    'daytrading_buying_power': '332807.56',
    'effective_buying_power': '332807.56',
    'equity': '105332.57',
    'id': '647fb3b4-0c43-4547-b847-3ce26d587404',
    'initial_margin': '36884.47',
    'intraday_adjustments': '0',
    'last_equity': '105332.57',
    'last_maintenance_margin': '22130.68',
    'long_market_value': '0',
    'maintenance_margin': '22130.68',
    'multiplier': '4',
    'non_marginable_buying_power': '68448.1',
    'options_approved_level': 3,
    'options_buying_power': '83201.89',
    'options_trading_level': 3,
    'pattern_day_trader': True,
    'pending_reg_t

In [6]:
# Setup Strategy and Model

# Set stock tickers for "MAG 6"
tickers = ["GOOGL", "AAPL", "AMZN", "META", "MSFT", "NVDA"]

# Specify date range and timeframe
start_date = '2022-01-01'
end_date = '2025-03-19'
timeframe = "1Day"

In [7]:
# Retrieve the historical data from Alpaca
all_data = {}
for ticker in tickers:
    barset = api.get_bars(
        ticker,
        timeframe,
        start=start_date,
        end=end_date
    )
    
    # Convert to pandas DataFrame
    df = barset.df
    
    # Ensure 'symbol' is a column in the DataFrame
    if 'symbol' in df.columns:
        df_ticker = df[df['symbol'] == ticker].copy()
    else:
        df_ticker = df.copy()  # If 'symbol' column doesn't exist, just copy it as is
    
    # Reset the index
    df_ticker = df_ticker.reset_index()
    
    # Store the cleaned DataFrame
    all_data[ticker] = df_ticker

In [8]:
# Add a 'symbol' column to each DataFrame, then concatenate
combined_df = pd.concat(
    [df.assign(symbol=ticker) for ticker, df in all_data.items()],
    ignore_index=True
)

print("Combined DataFrame shape:", combined_df.shape)

Combined DataFrame shape: (4830, 9)


In [9]:
combined_df

,timestamp,close,high,low,trade_count,open,volume,vwap,symbol
0,2022-01-03 05:00:00+00:00,2899.83,2917.0200,2874.255,103433,2901.10,1596148,2899.839679,GOOGL
1,2022-01-04 05:00:00+00:00,2887.99,2929.6978,2874.325,102670,2907.92,1581249,2893.127038,GOOGL
2,2022-01-05 05:00:00+00:00,2755.50,2889.9875,2753.760,199567,2888.40,2744554,2800.050979,GOOGL
3,2022-01-06 05:00:00+00:00,2754.95,2798.8000,2731.170,137862,2739.97,2031022,2763.505273,GOOGL
4,2022-01-07 05:00:00+00:00,2740.34,2768.9700,2715.330,111515,2762.91,1654885,2742.091958,GOOGL
...,...,...,...,...,...,...,...,...,...
4825,2025-03-13 04:00:00+00:00,115.58,117.7600,113.790,2246590,117.03,299033131,116.027528,NVDA
4826,2025-03-14 04:00:00+00:00,121.67,121.8800,118.150,1948613,118.61,277593455,120.592078,NVDA
4827,2025-03-17 04:00:00+00:00,119.53,122.8900,118.030,1950225,122.74,255501481,120.043459,NVDA
4828,2025-03-18 04:00:00+00:00,115.43,119.0200,114.540,2211483,118.00,299686944,116.453650,NVDA


In [10]:
# Count number of observations per symbol
nobs = combined_df.groupby("symbol").size()

# Filter symbols with more than 2 years of daily data (approx. 252 * 2 = 504 days)
mask = nobs[nobs > 2 * 12 * 21].index  # 2 years of trading months (21 days/month)

# Keep only symbols passing this threshold
filtered_df = combined_df[combined_df.symbol.isin(mask)].copy()
print(f"Symbols passing 2-year threshold: {list(mask)}") # Should be all chosen tickers

Symbols passing 2-year threshold: ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA']


In [11]:
# Create multi-index dataframe

# Sort by symbol & timestamp for proper time series alignment
filtered_df = filtered_df.sort_values(by=["symbol", "timestamp"])

# Keep only essential columns for momentum
# (assuming columns: 'timestamp', 'close', etc.)
prices = (
    filtered_df
    .set_index(["symbol", "timestamp"])  # directly set both columns as index
    .sort_index()                        # sort by that multi-level index
    [["close"]]                          # keep only the 'close' column
    .drop_duplicates()
)

print("Prices DataFrame shape:", prices.shape)

Prices DataFrame shape: (4488, 1)


In [12]:
prices

close
symbol timestamp                        
AAPL   2022-01-03 05:00:00+00:00  182.01
       2022-01-04 05:00:00+00:00  179.70
       2022-01-05 05:00:00+00:00  174.92
       2022-01-06 05:00:00+00:00  172.00
       2022-01-07 05:00:00+00:00  172.17
...                                  ...
NVDA   2025-03-13 04:00:00+00:00  115.58
       2025-03-14 04:00:00+00:00  121.67
       2025-03-17 04:00:00+00:00  119.53
       2025-03-18 04:00:00+00:00  115.43
       2025-03-19 04:00:00+00:00  117.52

[4488 rows x 1 columns]

In [13]:
# Rolling Momentum Factor
def momentum(close):
    returns = close.pct_change().iloc[-126:]
    factor = (((close.iloc[-21] - close.iloc[-252]) / close.iloc[-252]) - ((close.iloc[-1] - close.iloc[-21]) 
              / close.iloc[-21])) / np.std(returns)
    return factor

In [14]:
df = (
    prices
    .groupby("symbol", group_keys=False)   # group by each symbol
    .rolling(window=252)["close"]          # rolling window of 252 for each symbol
    .apply(momentum)                       # apply your custom function
)
df.index = df.index.droplevel(0)

In [15]:
prices["momentum"] = df
prices.dropna(inplace=True)
prices

close   momentum
symbol timestamp                                   
AAPL   2023-01-18 05:00:00+00:00  135.21 -11.712594
       2023-01-19 05:00:00+00:00  135.27 -12.595840
       2023-01-20 05:00:00+00:00  137.87 -12.592439
       2023-01-23 05:00:00+00:00  141.11 -11.192941
       2023-01-24 05:00:00+00:00  142.53 -13.704968
...                                  ...        ...
NVDA   2025-03-13 04:00:00+00:00  115.58 -17.068296
       2025-03-14 04:00:00+00:00  121.67 -19.577509
       2025-03-17 04:00:00+00:00  119.53 -20.282903
       2025-03-18 04:00:00+00:00  115.43 -20.402531
       2025-03-19 04:00:00+00:00  117.52 -20.291852

[2982 rows x 2 columns]

In [16]:
prices["factor_rank"] = (
    prices
    .groupby(level=[1])
    .momentum
    .rank(ascending=False)
)

In [17]:
prices

close   momentum  factor_rank
symbol timestamp                                                
AAPL   2023-01-18 05:00:00+00:00  135.21 -11.712594          3.0
       2023-01-19 05:00:00+00:00  135.27 -12.595840          3.0
       2023-01-20 05:00:00+00:00  137.87 -12.592439          3.0
       2023-01-23 05:00:00+00:00  141.11 -11.192941          2.0
       2023-01-24 05:00:00+00:00  142.53 -13.704968          3.0
...                                  ...        ...          ...
NVDA   2025-03-13 04:00:00+00:00  115.58 -17.068296          4.0
       2025-03-14 04:00:00+00:00  121.67 -19.577509          6.0
       2025-03-17 04:00:00+00:00  119.53 -20.282903          6.0
       2025-03-18 04:00:00+00:00  115.43 -20.402531          6.0
       2025-03-19 04:00:00+00:00  117.52 -20.291852          6.0

[2982 rows x 3 columns]

In [18]:
last_date = prices.index.get_level_values(1).max()
last_date

Timestamp('2025-03-19 04:00:00+0000', tz='UTC')

In [19]:
top_N = 3

In [20]:
stocks_to_buy = (
    prices
    .xs(last_date, level=1)
    .sort_values("factor_rank")
    .head(top_N)
    .assign(side=1) # 1 for buy
)

In [21]:
stocks_to_buy

,close,momentum,factor_rank,side
symbol,,,,
META,584.06,33.345299,1.0,1
AAPL,215.24,30.219164,2.0,1
AMZN,195.54,26.600390,3.0,1


In [22]:
stocks_to_short = (
    prices
    .xs(last_date, level=1)
    .sort_values("factor_rank")
    .tail(top_N)
    .assign(side=-1) # stocks to sell
)
stocks_to_short

,close,momentum,factor_rank,side
symbol,,,,
GOOGL,163.89,20.994410,4.0,-1
MSFT,387.82,4.266020,5.0,-1
NVDA,117.52,-20.291852,6.0,-1


In [23]:
stocks_to_trade = pd.concat([stocks_to_buy, stocks_to_short])
stocks_to_trade

,close,momentum,factor_rank,side
symbol,,,,
META,584.06,33.345299,1.0,1
AAPL,215.24,30.219164,2.0,1
AMZN,195.54,26.600390,3.0,1
GOOGL,163.89,20.994410,4.0,-1
MSFT,387.82,4.266020,5.0,-1
NVDA,117.52,-20.291852,6.0,-1


In [24]:
# Get current portfolio value
account_info = api.get_account()

# Print some relevant fields
print("Account ID:", account_info.id)
print("Account Status:", account_info.status)
print("Equity:", account_info.equity)
print("Buying Power:", account_info.buying_power)
print("Portfolio Value:", account_info.portfolio_value)


Account ID: 647fb3b4-0c43-4547-b847-3ce26d587404
Account Status: ACTIVE
Equity: 99924.06
Buying Power: 199848.12
Portfolio Value: 99924.06


In [25]:
# Convert the portfolio_value to float
port_val = float(account_info.portfolio_value)
print("Current Portfolio Value (USD):", port_val)


Current Portfolio Value (USD): 99924.06


In [26]:
# Assign an 'amount' column based on your side (buy=1, short=-1):
stocks_to_trade["amount"] = (
    (1 / top_N)
    * port_val*(0.5)
    * stocks_to_trade.side  # 1 for buy, -1 for short
)

stocks_to_trade

,close,momentum,factor_rank,side,amount
symbol,,,,,
META,584.06,33.345299,1.0,1,16654.01
AAPL,215.24,30.219164,2.0,1,16654.01
AMZN,195.54,26.600390,3.0,1,16654.01
GOOGL,163.89,20.994410,4.0,-1,-16654.01
MSFT,387.82,4.266020,5.0,-1,-16654.01
NVDA,117.52,-20.291852,6.0,-1,-16654.01


In [27]:
# Retrieve all open positions on Alpaca
positions = api.list_positions()

# Convert positions to a list of symbols
current_positions = [p.symbol for p in positions]
print("Currently held symbols:", current_positions)

Currently held symbols: []


In [28]:
divest_ = list(set(positions) - set(stocks_to_trade.index))
# Create a DataFrame for the divested symbols with zero amounts
divest = pd.DataFrame(
    index=divest_,
    data=np.zeros(len(divest_)),
    columns=["amount"]
)
divest

,amount


In [29]:
trade_amounts = pd.concat([divest, stocks_to_trade[["amount"]]])
trade_amounts

,amount
META,16654.01
AAPL,16654.01
AMZN,16654.01
GOOGL,-16654.01
MSFT,-16654.01
NVDA,-16654.01


In [30]:
import math

for symbol, row in trade_amounts.iterrows():
    target_dollars = row["amount"]
    
    # 1) If target_dollars == 0 => close the position
    if target_dollars == 0:
        if symbol in current_positions and current_positions[symbol] != 0:
            qty_held = current_positions[symbol]
            side = "sell" if qty_held > 0 else "buy"
            qty = abs(int(qty_held))
            print(f"Closing out {symbol}: {qty} shares, side={side.upper()}")
            api.submit_order(
                symbol=symbol,
                qty=qty,
                side=side,
                type="market",
                time_in_force="gtc"
            )
        else:
            print(f"No action needed for {symbol}, already at 0.")
    
    # 2) If target_dollars != 0 => compute share quantity
    else:
        # a) get last price
        latest_trade = api.get_latest_trade(symbol)
        if not latest_trade:
            print(f"No trade data found for {symbol}; skipping.")
            continue
        last_price = latest_trade.price
        
        # b) compute approximate shares to nearest whole share
        desired_shares = int(round(abs(target_dollars) / last_price))
        
        # c) side = buy if target_dollars > 0, sell if < 0
        side = "buy" if target_dollars > 0 else "sell"
        
        # d) skip if shares computed is 0
        if desired_shares == 0:
            print(f"Target dollars {target_dollars:.2f} for {symbol} is < 1 share at price {last_price:.2f}. Skipping.")
            continue
        
        print(f"{symbol}: {side.upper()} {desired_shares} shares at ~${last_price:.2f}")
        api.submit_order(
            symbol=symbol,
            qty=desired_shares,
            side=side,
            type="market",
            time_in_force="gtc"  # or "day", your preference
        )

META: BUY 28 shares at ~$594.61
AAPL: BUY 77 shares at ~$215.24
AMZN: BUY 85 shares at ~$195.85
GOOGL: SELL 102 shares at ~$163.09
MSFT: SELL 43 shares at ~$386.24
NVDA: SELL 142 shares at ~$117.37


In [62]:
orders = api.list_orders(status="all")
for o in orders:
    print(o.symbol, o.qty, o.side, o.status, o.filled_qty)

NVDA 142 sell filled 142
MSFT 43 sell filled 43
GOOGL 102 sell filled 102
AMZN 85 buy filled 85
AAPL 77 buy filled 77
META 28 buy filled 28
AAPL 155 sell filled 155
AMZN 170 sell filled 170
MSFT 86 buy filled 86
GOOGL 204 buy filled 204
META 56 sell filled 56
MSFT 86 sell filled 86
GOOGL 204 sell filled 204
AMZN 170 buy filled 170
AAPL 155 buy filled 155
META 56 buy filled 56
MSFT 86 buy filled 86
META 56 sell filled 56
AAPL 155 sell filled 155
AMZN 170 sell filled 170
GOOGL 204 buy filled 204
MSFT 86 sell filled 86
GOOGL 204 sell filled 204
AMZN 170 buy filled 170
AAPL 155 buy filled 155
META 56 buy filled 56
META 112.098773119 sell filled 112.098773119
AAPL 310.125325763 sell filled 310.125325763
AMZN 170.537859408 sell filled 170.537859408
AAPL 155 buy filled 155
META 56 buy filled 56
AMZN None buy filled 170.537859408
AAPL None buy filled 155.125325763
META None buy filled 56.098773119
